In [1]:
# check for requirements.txt

import os
import sys
import argparse

import torch
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import FashionMNIST
from torchvision import transforms # callable
from tqdm import tqdm

# from torchUtils import * 
# from models import *
# from dataloader import * 
# Users created package

In [2]:
# check if GPu is available
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(torch.cuda.is_available())

True


In [3]:
# Preprocessing & Dataset definition
data_root = os.path.join(os.getcwd(),"data")
# Composes several transforms together.
transform = transforms.Compose( 
    [
    transforms.ToTensor(), # PIL Image or ndarray-> FloatTensor with (c,h,w), Intensity to [0,1]
    transforms.Normalize([0.5],[0.5]) # normalize each channel of the input
    # mean (sequence) – Sequence of means for each channel.
    # std (sequence) – Sequence of standard deviations for each channel.
    ]
)
fashion_mnist_dataset = FashionMNIST(data_root, download=True, train=True, transform = transform)

In [4]:
data = random_split(fashion_mnist_dataset, [int(len(fashion_mnist_dataset)*0.9),int(len(fashion_mnist_dataset)*0.1)])
train_dataset = data[0]
val_dataset = data[1]

train_batchszie = 100
val_batchszie = 10

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batchszie, shuffle =True, num_workers=1
)
# Iterable object composed of iterable data

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batchszie, shuffle =True, num_workers=1
)


In [5]:
for sample_batch in train_dataloader:
    print(sample_batch[0].shape)
    break

torch.Size([100, 1, 28, 28])


In [6]:
class MLP(nn.Module):
    def __init__(self, input_dim: int, h1_dim: int, h2_dim: int, output_dim: int):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,h1_dim)
        self.linear2 = nn.Linear(h1_dim,h2_dim)
        self.linear3 = nn.Linear(h2_dim,output_dim)
        self.relu = F.relu
        
    def forward(self,input):
        x = torch.flatten(input,start_dim=1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        output = self.linear3(x)
        return output 

In [7]:
model = MLP(28*28,128,64,10)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
max_epoch=15
log_interval = 100

In [ ]:
train_step = 0
for epoch in range(1,max_epoch+1):
    
    # valid step
    with torch.no_grad():
        val_loss = 0.0
        val_corrects = 0
        
        for val_batch_idx, (val_images,val_labels) in enumerate(tqdm(val_dataloader,position=0,leave=True,desc='validation')):
            val_outputs = model(val_images)
            _, val_preds = torch.max(val_outputs,1)
            val_loss += loss_function(val_outputs,val_labels) / val_outputs.shape[0] # per batch
            val_corrects += torch.sum(val_preds == val_labels.data) / val_outputs.shape[0] # per batch
    
    # Validation logging
    val_epoch_loss = val_loss / len(val_dataloader)
    val_epoch_acc = val_corrects / len(val_dataloader)
    
    print(
        f"{epoch}: epoch, {train_step} step: val_loss : {val_epoch_loss}, val_acc: {val_epoch_acc}"
    )
    
    # train step
    for batch_idx, (images,labels) in enumerate(tqdm(train_dataloader,position=0,leave=True,desc='validation')):
        current_loss = 0.0
        current_corrects = 0
        
        #Forward
        outputs = model(images)
        _, preds = torch.max(outputs,1)
        loss = loss_function(outputs,labels)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        current_loss += loss.item()
        current_corrects += torch.sum(preds==labels.data)
        # Train logging
        if train_step % log_interval == 0: # Performance log per batch
            train_loss = current_loss / log_interval # average loss
            train_acc = current_corrects / log_interval # average acc
            print(
                f"{train_step}: train_loss : {train_loss}, train_acc: {train_acc}"
            )
            current_loss = 0
            current_corrects = 0
            
        train_step+=1
        

validation: 100%|███████████████████████████████████████████████████████████████████| 600/600 [00:01<00:00, 301.85it/s]


1: epoch, 0 step: val_loss : 0.2319396436214447, val_acc: 0.03933339938521385


validation:   2%|█▎                                                                   | 10/540 [00:01<00:43, 12.23it/s]

0: train_loss : 0.023045382499694823, train_acc: 0.019999999552965164


validation:  21%|██████████████▌                                                     | 116/540 [00:02<00:05, 77.28it/s]

...

validation:  69%|██████████████████████████████████████████████▊                     | 372/540 [00:05<00:02, 77.48it/s]

900: train_loss : 0.004266084432601929, train_acc: 0.8199999928474426


validation:  87%|██████████████████████████████████████████████████████████▉         | 468/540 [00:06<00:00, 78.79it/s]

1000: train_loss : 0.0033033570647239686, train_acc: 0.8999999761581421


validation:  54%|███████████████████████████████████▊                               | 321/600 [00:01<00:00, 406.63it/s]